# Introdução ao Processamento de Linguagem Natural (PLN) e Pré-processamento de Texto

Este notebook tem como objetivo introduzir os conceitos fundamentais de Processamento de Linguagem Natural (PLN) e, em particular, as técnicas essenciais de pré-processamento de texto. O pré-processamento é uma etapa crucial em praticamente qualquer tarefa de PLN, pois transforma dados textuais brutos em um formato mais limpo e estruturado, adequado para análise e modelagem computacional.

Vamos cobrir:
1.  O que é PLN e por que o pré-processamento é importante.
2.  Principais etapas do pré-processamento de texto.
3.  Expressões Regulares (Regex)
4.  Stemming (Radicalização) vs. Lematização.
5.  Exemplos práticos de código utilizando a biblioteca NLTK e o módulo `re` em Python.
6.  Exercícios práticos para aplicar os conhecimentos.

---




# 1. O que é PLN e Por que Pré-processar Texto?

### O que é PLN?

**Processamento de Linguagem Natural (PLN)**, do inglês *Natural Language Processing (NLP)*, é um campo da inteligência artificial que se concentra na interação entre computadores e linguagens humanas (naturais). O objetivo é permitir que os computadores "entendam", "interpretem" e "manipulem" a linguagem humana.

Aplicações de PLN incluem:
*   Assistentes virtuais (Siri, Google Assistant, Alexa)
*   Tradutores automáticos (Google Translate)
*   Análise de Sentimento (entender a opinião em textos)
*   Chatbots
*   Sumarização automática de textos
*   Sistemas de busca (Google Search)

### Por que Pré-processar Texto?

A linguagem humana é rica, complexa e muitas vezes ambígua. Textos brutos contêm ruídos, variações gramaticais, erros de digitação, pontuação e formatação que podem dificultar o processamento computacional.

O **pré-processamento de texto** é o processo de limpeza e preparação dos dados textuais para que possam ser utilizados de forma eficaz em tarefas de PLN. É como limpar e organizar ingredientes antes de cozinhar; os resultados serão muito melhores!

**Objetivos do pré-processamento:**
*   Reduzir ruído e redundância.
*   Padronizar o texto.
*   Converter texto em um formato mais fácil de analisar (e.g., listas de palavras).
*   Reduzir a dimensionalidade dos dados (número de palavras únicas).

---

# 2. Principais Etapas do Pré-processamento de Texto

Vamos explorar algumas das etapas mais comuns e importantes no pré-processamento de texto. Usaremos a biblioteca NLTK (Natural Language Toolkit) em Python, que é uma das ferramentas mais populares para PLN, e o módulo `re` para Expressões Regulares.

Primeiro, precisamos instalar a biblioteca NLTK e baixar os recursos necessários.

In [1]:
import nltk
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
from collections import Counter
import itertools

# Baixar recursos necessários do NLTK
# punkt: para tokenização de sentenças e palavras
# stopwords: lista de palavras comuns a serem removidas
# rslp: stemmer para português
# wordnet: dicionário para lematização (principalmente inglês)
# averaged_perceptron_tagger: para POS tagging (usado na lematização)

print("Verificando e baixando recursos NLTK...")

nltk_resources = ['punkt', 'stopwords', 'rslp', 'wordnet', 'averaged_perceptron_tagger']
for resource in nltk_resources:
    try:
        nltk.data.find(f'tokenizers/{resource}' if resource == 'punkt' else f'corpora/{resource}' if resource in ['stopwords', 'wordnet'] else f'stemmers/{resource}' if resource == 'rslp' else f'taggers/{resource}')
        print(f"Recurso '{resource}' já baixado.")
    except LookupError:
         print(f"Recurso '{resource}' não encontrado localmente. Baixando...")
         nltk.download(resource)


print("Verificação e download de recursos NLTK concluídos!")

# Carregar stop words do NLTK e stemmer (para uso posterior)
stop_words_portugues = set(stopwords.words('portuguese'))
stemmer_portugues = RSLPStemmer()

Verificando e baixando recursos NLTK...
Recurso 'punkt' já baixado.
Recurso 'stopwords' já baixado.
Recurso 'rslp' já baixado.
Recurso 'wordnet' não encontrado localmente. Baixando...
Recurso 'averaged_perceptron_tagger' já baixado.
Verificação e download de recursos NLTK concluídos!


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
texto_exemplo = "Este É; 1 dos 30 eXEMplos - de TeXTO - COM 5 DIferentes... CAPitaliZAçÕES!\n"

## a) Converter para Minúsculas (Lowercasing)

Converter todo o texto para minúsculas ajuda a garantir que palavras que diferem apenas na capitalização sejam tratadas como a mesma palavra (ex: "Palavra", "palavra", "PALAVRA").

In [3]:
texto_minusculo = texto_exemplo.lower()

print(f"Texto original: {texto_exemplo}")
print(f"Texto em minúsculas: {texto_minusculo}")

Texto original: Este É; 1 dos 30 eXEMplos - de TeXTO - COM 5 DIferentes... CAPitaliZAçÕES!

Texto em minúsculas: este é; 1 dos 30 exemplos - de texto - com 5 diferentes... capitalizações!



##b) Remover Pontuação
Pontuações (vírgulas, pontos, pontos de exclamação, etc.) geralmente não carregam significado semântico para a maioria das tarefas de PLN e podem ser removidas.

In [4]:
texto_despontuado = texto_minusculo.translate(str.maketrans('', '', string.punctuation))

print(f"Texto original: {texto_minusculo}")
print(f"Texto sem pontuação: {texto_despontuado}")

Texto original: este é; 1 dos 30 exemplos - de texto - com 5 diferentes... capitalizações!

Texto sem pontuação: este é 1 dos 30 exemplos  de texto  com 5 diferentes capitalizações



## c) Remover Números
Dependendo da tarefa, números podem ser irrelevantes. Já mostramos um exemplo usando o módulo re (Expressões Regulares) para isso. Veremos mais sobre Regex em detalhes a seguir.

A remoção de números (`re.sub(r'\d+', '', text)`) é uma etapa comum em muitos pipelines de pré-processamento, mas a decisão de fazê-lo **depende totalmente da sua tarefa de PLN**.

**Razões Comuns para Remover Números:**

1.  **Reduzir Ruído e Dimensionalidade:** Para tarefas como análise de sentimento geral ou modelagem de tópicos, o valor numérico exato (como "100", "2023", "5") muitas vezes não contribui para o *significado principal* da frase. Um review que diz "O produto custa 100 reais" e outro que diz "O produto custa 200 reais" podem ter o mesmo sentimento ("neutro" ou "positivo/negativo" dependendo do contexto da frase). Tratar "100" e "200" como tokens separados pode inchar seu vocabulário sem adicionar valor semântico relevante para *essa tarefa específica*.
2.  **Focar no Conteúdo Textual:** Muitos algoritmos de PLN focam na relação e frequência das palavras. Números podem quebrar padrões de texto ("comprou em [NÚMERO] e gostou") sem que o número em si seja o foco da análise.
3.  **Padronização:** Em alguns casos, você pode querer saber *que* um número estava presente, mas não qual era. Substituir todos os números por um token especial (como `<NUM>`) padroniza essa informação e ainda reduz a dimensionalidade (em vez de infinitos números possíveis, você tem apenas um token `<NUM>`).

**Quando NÃO Remover Números (ou Tratá-los Diferente):**

Você **NÃO DEVE** remover números se a informação numérica for crucial para a tarefa de PLN. Exemplos:

*   **Análise de Reviews de Produtos/Apps:** Se a tarefa for analisar *por que* um produto recebeu 1 estrela vs. 5 estrelas, o número na frase "Recebi 1 estrela mas o produto é ótimo" ou "Dou 5 estrelas" é *extremamente* importante.
*   **Análise Financeira:** Valores monetários ("lucro de 1 milhão"), percentuais, datas, etc., são o cerne da análise.
*   **Extração de Informação:** Se você quer extrair endereços, números de telefone, datas de validade, versões de software ("versão 3.0"), remover números inviabilizaria a tarefa.
*   **Análise de Textos Científicos/Técnicos:** Medições, fórmulas, códigos de produtos, etc., são essenciais.

In [5]:
texto_desnumerado = re.sub(r'\d+', '', texto_despontuado)
# r'\d+' <- padrão de expressão regular para busca de numeros

print(f"Texto original: {texto_despontuado}")
print(f"Texto sem números: {texto_desnumerado}")

Texto original: este é 1 dos 30 exemplos  de texto  com 5 diferentes capitalizações

Texto sem números: este é  dos  exemplos  de texto  com  diferentes capitalizações



## d) Remover Espaços em Branco Extras
Múltiplos espaços entre palavras ou espaços no início/fim da string podem ser limpos. Uma forma simples é usar ``split()`` e ``join()``. Outra forma muito eficiente é usar Expressões Regulares, como veremos adiante.

In [6]:
texto_sem_espacos = ' '.join(texto_desnumerado.split())

print(f"Texto original: {texto_desnumerado}")
print(f"Texto sem espaços extras: {texto_sem_espacos}")

Texto original: este é  dos  exemplos  de texto  com  diferentes capitalizações

Texto sem espaços extras: este é dos exemplos de texto com diferentes capitalizações


---

# 3. Expressões Regulares (Regex) como aliadas no pré-processamento

**Expressões Regulares (Regex ou Regexp)** são sequências de caracteres que definem um *padrão de busca*. São incrivelmente poderosas para encontrar, substituir ou extrair padrões complexos em texto de forma concisa. Embora a sintaxe possa parecer intimidadora no início, dominar o básico de Regex é um superpoder no processamento de texto.

Em Python, o módulo `re` oferece suporte nativo para Expressões Regulares.

### Por que usar Regex em PLN?

*   **Limpeza de Ruído:** Remover padrões indesejados como tags HTML, URLs, emails, caracteres especiais, múltiplos espaços.
*   **Extração de Informação:** Encontrar números de telefone, datas, menções de usuários, hashtags, etc.
*   **Validação de Formato:** Verificar se uma string segue um formato específico (ex: formato de email, CPF).
*   **Substituição:** Substituir padrões encontrados por outro texto (como fizemos para remover números).

### Sintaxe Básica de Regex

Aqui estão alguns dos elementos de padrão mais comuns:

*   `a`, `1`, `@` - Caracteres Literais: correspondem exatamente a eles mesmos.
*   `.` (Ponto): Corresponde a qualquer caractere (exceto quebra de linha `\n`).
*   `*` (Asterisco): Corresponde a **zero ou mais** ocorrências do elemento anterior. Ex: `a*` corresponde a "", "a", "aa", "aaa"...
*   `+` (Mais): Corresponde a **uma ou mais** ocorrências do elemento anterior. Ex: `a+` corresponde a "a", "aa", "aaa"... mas não "".
*   `?` (Interrogação): Corresponde a **zero ou uma** ocorrência do elemento anterior. Ex: `a?` corresponde a "" ou "a".
*   `|` (Barra Vertical): Operador **OR**. Ex: `gato|cachorro` corresponde a "gato" ou "cachorro".
*   `()` (Parênteses): Agrupamento e Captura. Permite aplicar operadores (`*`, `+`, `?`) a um grupo inteiro e capturar partes da string.
*   `[]` (Colchetes): Conjunto de Caracteres. Corresponde a **qualquer um** dos caracteres dentro dos colchetes. Ex: `[aeiou]` corresponde a qualquer vogal minúscula. `[0-9]` corresponde a qualquer dígito. `[a-zA-Z]` corresponde a qualquer letra (maiúscula ou minúscula).
*   `-` (Hífen) dentro de `[]`: Define um Intervalo. Ex: `[a-z]` (letras minúsculas de a a z), `[0-9]` (dígitos de 0 a 9).
*   `^` (Circunflexo) dentro de `[]`: Negação. Corresponde a **qualquer caractere que NÃO ESTÁ** no conjunto. Ex: `[^0-9]` corresponde a qualquer caractere que não seja um dígito.
*   `\` (Barra Invertida): Caractere de Escape. Usado para "escapar" caracteres especiais (como `.`, `*`, `+`, `?`, `|`, `(`, `)`, `[`, `]`, `{`, `}`, `^`, `$`, `\` ) ou para introduzir sequências especiais.
    *   `\d`: Corresponde a um dígito (`[0-9]`).
    *   `\D`: Corresponde a um não-dígito (`[^0-9]`).
    *   `\w`: Corresponde a um caractere de "palavra" (`[a-zA-Z0-9_]`).
    *   `\W`: Corresponde a um não-caractere de "palavra" (`[^a-zA-Z0-9_]`).
    *   `\s`: Corresponde a um caractere de espaço em branco (espaço, tab `\t`, quebra de linha `\n`, retorno de carro `\r`, feed de formulário `\f`, tab vertical `\v`).
    *   `\S`: Corresponde a um não-caractere de espaço em branco.
*   `^` (Circunflexo) fora de `[]`: Corresponde ao **início** da string.
*   `$` (Cifrão): Corresponde ao **fim** da string.



**Strings Raw (Brutas) em Python (`r"..."`)**: É uma boa prática usar strings raw para padrões Regex em Python (prefixo `r`). Isso impede que as barras invertidas (`\`) sejam interpretadas pelo Python como sequências de escape de string Python (ex: `\n` para nova linha) antes de serem passadas para o motor de Regex.

### Funções do Módulo `re` (Principais para Pré-processamento)

*   `re.search(pattern, string)`: Procura pela *primeira* ocorrência do padrão na string. Retorna um objeto `Match` se encontrar, `None` caso contrário.
*   `re.findall(pattern, string)`: Encontra *todas* as ocorrências não sobrepostas do padrão na string e retorna uma **lista** de strings correspondentes.
*   `re.sub(pattern, repl, string)`: Substitui todas as ocorrências do `pattern` encontrado na `string` pela string `repl`. É a função mais usada para limpeza.

Vamos ver alguns exemplos práticos:

In [7]:
texto_regex = "Meu email é teste@example.com e meu site é https://www.site.org.    Meu telefone é (11) 98765-4321.    Há também números 12345 e símbolos !!!"

print(f"Texto original: {texto_regex}\n")

Texto original: Meu email é teste@example.com e meu site é https://www.site.org.    Meu telefone é (11) 98765-4321.    Há também números 12345 e símbolos !!!



In [8]:
# Exemplo 1: Encontrar todos os números

print("Números encontrados:")
numeros = re.findall(r'\d+', texto_regex)
print(numeros)

Números encontrados:
['11', '98765', '4321', '12345']


In [9]:
# Exemplo 2: Substituir todos os espaços em branco múltiplos por um único espaço
# \s+ significa "um ou mais caracteres de espaço em branco"

print("Texto com espaços em branco substituídos:")
texto_sem_espaco = re.sub(r'\s+', ' ', texto_regex)
print(texto_sem_espaco)

Texto com espaços em branco substituídos:
Meu email é teste@example.com e meu site é https://www.site.org. Meu telefone é (11) 98765-4321. Há também números 12345 e símbolos !!!


In [10]:
# Exemplo 3: Remover caracteres que não são letras, números ou espaços
# [^\w\s] significa "qualquer caractere que NÃO SEJA (\^) um caractere de palavra (\w) OU (\s) um espaço em branco"
# Poderíamos substituir por um espaço ou uma string vazia
texto_com_especiais = "Texto! Com? Simbolos@ E. Pontuacao,"

print("Texto com caracteres especiais removidos:")
texto_sem_especiais = re.sub(r'[^\w\s]', '', texto_com_especiais)
print(texto_sem_especiais)

Texto com caracteres especiais removidos:
Texto Com Simbolos E Pontuacao


In [11]:
# Exemplo 4: Remover URLs (exemplo comum em reviews online)
texto_com_url = "Produto bom! Veja em: http://loja.com/produto1 ou https://outraloja.org"

print("Texto com URLs removidas:")
texto_sem_url = re.sub(r'https?://\S+', '', texto_com_url)
print(texto_sem_url)

Texto com URLs removidas:
Produto bom! Veja em:  ou 


Como visto, Regex é uma ferramenta flexível que pode complementar ou substituir algumas das etapas de limpeza manuais que vimos anteriormente. No nosso pipeline de pré-processamento (seção 2), já usamos `re.sub` para remover números. Poderíamos usá-la também para remover pontuação ou limpar espaços.

# 4. Continuação das Etapas de Pré-processamento

Agora que temos uma ferramenta poderosa para limpeza de padrões, vamos continuar com as etapas mais focadas na estrutura e significado das palavras.

### e) Tokenização

Tokenização é o processo de dividir uma string de texto em pedaços menores chamados **tokens**. Os tokens podem ser palavras, sub-palavras, ou até mesmo frases, dependendo da granularidade desejada. Geralmente, tokenizamos em palavras.

In [12]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [13]:
texto_para_tokenizar = "Tokenização é o processo de dividir texto em palavras."
tokens = word_tokenize(texto_para_tokenizar, language='portuguese')

print(f"Texto original: {texto_para_tokenizar}")
print(f"Tokens: {tokens}")

Texto original: Tokenização é o processo de dividir texto em palavras.
Tokens: ['Tokenização', 'é', 'o', 'processo', 'de', 'dividir', 'texto', 'em', 'palavras', '.']


### f) Remover Stop Words

**Stop words** são palavras muito comuns em uma língua que geralmente não adicionam muito significado a uma frase (ex: "de", "a", "o", "e", "é", "em", "para", etc.). Removê-las pode reduzir o tamanho do vocabulário e melhorar o desempenho em algumas tarefas.

O NLTK possui listas de stop words para vários idiomas.

In [14]:
texto_com_stopwords = "Esta é uma frase de exemplo com muitas palavras comuns e irrelevantes."
tokens = word_tokenize(texto_com_stopwords, language='portuguese')

# Convertendo tokens para minúsculas ANTES de verificar se são stop words
tokens_sem_stopwords = [palavra for palavra in tokens if palavra.lower() not in stop_words_portugues]

print(f"Tokens originais: {tokens}")
print(f"Stop words em português (exemplo): {list(stop_words_portugues)[:10]}...") # Mostrar apenas as primeiras 10
print(f"Tokens sem stop words: {tokens_sem_stopwords}")

Tokens originais: ['Esta', 'é', 'uma', 'frase', 'de', 'exemplo', 'com', 'muitas', 'palavras', 'comuns', 'e', 'irrelevantes', '.']
Stop words em português (exemplo): ['pelas', 'um', 'éramos', 'meu', 'só', 'seus', 'esta', 'estas', 'houvemos', 'teriam']...
Tokens sem stop words: ['frase', 'exemplo', 'muitas', 'palavras', 'comuns', 'irrelevantes', '.']


# 5. Stemming (Radicalização) vs. Lematização

Essas duas técnicas têm um objetivo semelhante: reduzir palavras flexionadas ou derivadas para uma forma base. No entanto, a abordagem e o resultado são diferentes.

### Stemming (Radicalização)

**Stemming** é um processo heurístico que corta sufixos (e às vezes prefixos) das palavras para chegar a um "radical" (stem). O radical resultante **não necessariamente é uma palavra real** da língua. É um processo mais rápido e menos preciso, útil para agrupar palavras com significados semelhantes.

Exemplo em inglês: "running", "runs", "ran" podem ser reduzidos para o radical "run". "beautiful", "beauty" podem ser reduzidos para "beauti".

Para português, o NLTK oferece o RSLP Stemmer (Remoção de Sufixos da Língua Portuguesa).

In [15]:
# stemmer_portugues já carregado no início

palavras = ["correndo", "correram", "corre", "bonito", "beleza", "mulheres", "mulher", "linguagem", "linguagens"]


# Note que alguns radicais ("corr", "mulher") são palavras reais,
# mas outros ("bonit", "linguag") não são necessariamente.

**Lematização** é um processo mais sofisticado que utiliza vocabulário (um dicionário de palavras e suas formas) e análise morfológica para reduzir palavras flexionadas à sua forma base ou dicionário, conhecida como **lema**. O lema resultante **sempre é uma palavra real** da língua. É um processo mais lento, mas geralmente mais preciso que o stemming.

Exemplo em inglês: "running", "runs", "ran" são lematizados para o lema "run". "better" é lematizado para "good". "am", "is", "are" são lematizados para "be".

O NLTK possui o `WordNetLemmatizer`, mas ele funciona melhor para inglês e requer a especificação da Parte da Fala (POS tag) da palavra para um resultado mais preciso. Lematização robusta para português geralmente requer bibliotecas mais avançadas como spaCy ou outras ferramentas específicas para português, pois o NLTK WordNet não cobre bem o vocabulário português.

Vamos demonstrar o conceito com o lemmatizer do NLTK para inglês e discutir o caso do português.

In [16]:
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [17]:
from nltk.stem import WordNetLemmatizer
# Para lematizar corretamente, muitas vezes precisamos do POS tag (Parte da Fala)
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

# Função auxiliar para obter o POS tag no formato que o WordNetLemmatizer entende
def get_wordnet_pos(tag):
    """Converte o POS tag do NLTK no formato do WordNet"""
    if tag.startswith('J'): # Adjetivo
        return wordnet.ADJ
    elif tag.startswith('V'): # Verbo
        return wordnet.VERB
    elif tag.startswith('N'): # Substantivo
        return wordnet.NOUN
    elif tag.startswith('R'): # Advérbio
        return wordnet.ADV
    else:
        return wordnet.NOUN # Padrão para substantivo

In [18]:
# Exemplo em Inglês (para ilustrar o conceito com WordNetLemmatizer)
palavras_en = ["running", "runs", "ran", "better", "geese", "children", "is", "are"]

# Obter POS tags para cada palavra (simplificado para demonstração)
# Em um pipeline real, processaríamos a frase inteira com um POS tagger
pos_tags_en = nltk.pos_tag(palavras_en)

lemas_en = [
    lemmatizer.lemmatize(palavra, pos=get_wordnet_pos(tag))
    for palavra, tag in pos_tags_en
]

print(f"Palavras originais (Inglês): {palavras_en}")
print(f"POS Tags (Inglês - simplificado): {pos_tags_en}")
print(f"Lemas (Inglês): {lemas_en}")

Palavras originais (Inglês): ['running', 'runs', 'ran', 'better', 'geese', 'children', 'is', 'are']
POS Tags (Inglês - simplificado): [('running', 'VBG'), ('runs', 'NNS'), ('ran', 'VBD'), ('better', 'JJR'), ('geese', 'JJ'), ('children', 'NNS'), ('is', 'VBZ'), ('are', 'VBP')]
Lemas (Inglês): ['run', 'run', 'run', 'good', 'geese', 'child', 'be', 'be']


In [19]:
print("\n--- Lematização em Português ---")
print("Para português, WordNetLemmatizer do NLTK não é adequado, pois não cobre o vocabulário.")
print("Bibliotecas como spaCy (com modelo pt) ou recursos específicos para português são necessários.")
print("Exemplo conceitual de lemas esperados para português:")
palavras_pt = ["andando", "andaram", "anda", "feliz", "felicidade", "bonita", "bonitas"]
# Lema para "andando", "andaram", "anda" seria "andar"
# Lema para "feliz", "felicidade" seria "feliz"
# Lema para "bonita", "bonitas" seria "bonito" (ou "bonita" dependendo da convenção/recurso)

print(f"Palavras originais (Português): {palavras_pt}")
print("Lemas esperados (Conceito): ['andar', 'andar', 'andar', 'feliz', 'feliz', 'bonito', 'bonito']")

# Comparação com Stemming RSLP
# stemmer_portugues já carregado no início
radicais_pt = [stemmer_portugues.stem(palavra) for palavra in palavras_pt]
print(f"Radicais (Stemming RSLP): {radicais_pt}")

# Note a diferença: stemming produz radicais que podem não ser palavras reais ("and", "bonit"),
# enquanto a lematização busca a forma do dicionário ("andar", "feliz", "bonito").


--- Lematização em Português ---
Para português, WordNetLemmatizer do NLTK não é adequado, pois não cobre o vocabulário.
Bibliotecas como spaCy (com modelo pt) ou recursos específicos para português são necessários.
Exemplo conceitual de lemas esperados para português:
Palavras originais (Português): ['andando', 'andaram', 'anda', 'feliz', 'felicidade', 'bonita', 'bonitas']
Lemas esperados (Conceito): ['andar', 'andar', 'andar', 'feliz', 'feliz', 'bonito', 'bonito']
Radicais (Stemming RSLP): ['and', 'and', 'and', 'feliz', 'felic', 'bonit', 'bonit']


**Quando usar Stemming vs. Lematização?**

*   **Stemming:** Mais rápido, bom para tarefas onde a velocidade é importante e não é essencial que o resultado seja uma palavra real (ex: sistemas de busca simples onde você quer agrupar variações de uma palavra).
*   **Lematização:** Mais lento, mais preciso, bom para tarefas que exigem um entendimento mais profundo do significado da palavra ou onde a forma real da palavra é importante (ex: análise de sentimento, tradução automática, construção de vocabulário para modelos complexos).

---

# 6. Pipeline Completo de Pré-processamento

Vamos juntar várias dessas etapas em uma função para processar um texto de exemplo. Incorporaremos o uso de Regex na limpeza.

In [20]:
def preprocess_text(text, remove_numbers=True, remove_punctuation=True,
                      remove_stopwords=True, apply_stemming=False):
    """
    Aplica um pipeline de pré-processamento a um texto usando NLTK e Regex.

    Args:
        text (str): O texto de entrada.
        remove_numbers (bool): Se True, remove números usando Regex.
        remove_punctuation (bool): Se True, remove pontuação usando string.punctuation.
        remove_stopwords (bool): Se True, remove stop words (do NLTK).
        apply_stemming (bool): Se True, aplica stemming RSLP (do NLTK).

    Returns:
        list: Uma lista de tokens pré-processados.
    """
    if not isinstance(text, str):
        return [] # Retorna lista vazia para entradas não string

    # 1. Converter para minúsculas
    text = text.lower()

    # 2. Remover URLs, mentions, hashtags (exemplos comuns com Regex)
    text = re.sub(r'https?://\S+', '', text) # Remove URLs (http, https)
    text = re.sub(r'@\w+', '', text) # Remove mentions (@usuario)
    text = re.sub(r'#\w+', '', text) # Remove hashtags (#hashtag)


    # 3. Remover números (se solicitado) usando Regex
    if remove_numbers:
        text = re.sub(r'\d+', '', text)

    # 4. Remover pontuação (se solicitado) usando string.punctuation
    if remove_punctuation:
         text = text.translate(str.maketrans('', '', string.punctuation))

    # 5. Remover espaços em branco extras usando Regex (e strip)
    text = re.sub(r'\s+', ' ', text).strip()

    # Certificar-se de que o texto não está vazio após a limpeza inicial
    if not text:
        return []

    # 6. Tokenização (NLTK)
    tokens = word_tokenize(text, language='portuguese')

    # 7. Remover stop words (se solicitado - NLTK)
    if remove_stopwords:
        tokens = [palavra for palavra in tokens if palavra not in stop_words_portugues]

    # 8. Aplicar stemming (se solicitado - NLTK)
    if apply_stemming:
        tokens = [stemmer_portugues.stem(palavra) for palavra in tokens]

    # Opcional: Remover tokens vazios ou de 1 caractere que podem ter sobrado
    tokens = [token for token in tokens if token and len(token) > 1]

    return tokens

In [21]:
# Exemplo de uso da função
texto_completo_exemplo = "Olá mundo! Este é um teste completo para ver como o pré-processamento funciona com PALAVRAS, números 123 e pontuações diferentes! Estou testando o stemming e a lematização. Site: http://teste.com.br. Curti muito o app! #top @usuario"

print(f"Texto original: {texto_completo_exemplo}\n")

# --- Processamento com NLTK (sem stemming) ---
tokens_nltk_basico = preprocess_text(texto_completo_exemplo)
print(f"Tokens (NLTK): {tokens_nltk_basico}")

Texto original: Olá mundo! Este é um teste completo para ver como o pré-processamento funciona com PALAVRAS, números 123 e pontuações diferentes! Estou testando o stemming e a lematização. Site: http://teste.com.br. Curti muito o app! #top @usuario

Tokens (NLTK): ['olá', 'mundo', 'teste', 'completo', 'ver', 'préprocessamento', 'funciona', 'palavras', 'números', 'pontuações', 'diferentes', 'testando', 'stemming', 'lematização', 'site', 'curti', 'app']


In [22]:
# --- Processamento com NLTK (com stemming) ---
print("--- Processamento com NLTK (com stemming RSLP) ---")
tokens_nltk_steeming = preprocess_text(texto_completo_exemplo, apply_stemming=True)
print(f"Tokens (NLTK + Stemming): {tokens_nltk_steeming}")

--- Processamento com NLTK (com stemming RSLP) ---
Tokens (NLTK + Stemming): ['olá', 'mund', 'test', 'complet', 'ver', 'préprocess', 'func', 'palavr', 'númer', 'pontu', 'difer', 'test', 'stemming', 'lemat', 'sit', 'curt', 'app']


# 7. Exercícios Práticos: Raspando e Pré-processando Comentários de Apps

Agora é a sua vez de colocar a mão na massa! Vamos aplicar o que aprendemos raspando comentários de um aplicativo da Google Play Store e pré-processando-os.

Usaremos a biblioteca `google-play-scraper`. Você precisará instalá-la.

In [23]:
!pip install google-play-scraper

### **Exercício 1: Raspar Comentários de um App**

Escolha um aplicativo na Google Play Store. Você pode encontrar o ID do aplicativo na URL da página dele. Por exemplo, a URL do WhatsApp é `https://play.google.com/store/apps/details?id=com.whatsapp`. O ID é `com.whatsapp`.

Vamos raspar uma quantidade limitada de comentários para este exercício.

In [24]:
from google_play_scraper import Sort, reviews
import pandas as pd

# Substitua 'com.whatsapp' pelo ID do aplicativo que você quer raspar
APP_ID = 'com.google.android.youtube' # Exemplo: WhatsApp
NUM_REVIEWS = 20 # Quantidade de reviews a raspar (máx 200 por chamada com Sort.MOST_RELEVANT)

all_reviews = []

In [25]:
# A API scraper geralmente só retorna 200 por vez para a ordenação mais relevante
# Se quisesse mais, precisaria usar outra ordenação ou múltiplos tokens,
# mas para o exercício, 200 é suficiente.
try:
    result, continuation_token = reviews(
        APP_ID,
        lang='pt', # Idioma dos reviews (português)
        country='br', # País
        sort=Sort.MOST_RELEVANT, # Tipo de ordenação
        count=NUM_REVIEWS # Quantidade
    )
    all_reviews.extend(result)

    print(f"Raspagem concluída. Foram encontrados {len(all_reviews)} reviews.")

    if len(all_reviews) > 0:
        # Exibir os primeiros 5 reviews raspados
        print("\nPrimeiros 5 reviews:")
        for i, review in enumerate(all_reviews[:5]):
            print(f"--- Review {i+1} ---")
            print(f"Usuário: {review.get('userName', 'N/A')}")
            print(f"Pontuação: {review.get('score', 'N/A')}")
            print(f"Data: {review.get('at', 'N/A')}")
            print(f"Conteúdo: {review.get('content', 'N/A')}")
            print("-----------------")

        # Criar um DataFrame pandas com os reviews (útil para inspecionar, mas não essencial para o pré-processamento puro)
        reviews_df = pd.DataFrame(all_reviews)
        print(f"\nDataFrame criado com {len(reviews_df)} reviews.")
        # Extrair apenas o conteúdo dos reviews para pré-processamento
        review_contents = reviews_df['content'].dropna().tolist() # Remover reviews sem conteúdo e converter para lista

        print(f"\nTotal de conteúdos de reviews extraídos para processamento: {len(review_contents)}")
    else:
        print("Nenhum review encontrado para o APP_ID especificado.")
        reviews_df = pd.DataFrame()
        review_contents = [] # Lista vazia se não encontrar reviews

except Exception as e:
    print(f"Ocorreu um erro durante a raspagem dos reviews: {e}")
    reviews_df = pd.DataFrame()
    review_contents = [] # Garante que review_contents é uma lista mesmo em caso de erro

Raspagem concluída. Foram encontrados 20 reviews.

Primeiros 5 reviews:
--- Review 1 ---
Usuário: Jebs freire
Pontuação: 2
Data: 2025-05-27 14:30:32
Conteúdo: Tem algum Bug nos comentários, vc precisa enviar várias vezes, e é apagado. E não é nada proibido, até uma carinha de emoji feliz não fica, é apagado. O que é isso? Os anúncios são chatos, tem alguns que são longos. Quando é curto ainda dá pra aguentar, mas se vc não pula vem outro anúncio e outro e outro... E demorados. Além disso, está com um bug estranho quando avanço 10 seg, se vc tenta avançar mais, trava, parece um disco arranhado ou um dj. Arrumem isso!
-----------------
--- Review 2 ---
Usuário: Pikachu de Bigode
Pontuação: 4
Data: 2025-05-29 20:21:25
Conteúdo: Bom, aqui estou eu dnv, o problema do pop-up desde aquele tempo tem sido resolvido pelo q dá para ver, mas agr é uma sugestão, seria legal na aba de histórico acrescentar categorias, por exemplo: assisto um clipe de música, então ele fica na categoria de música; as

### **Exercício 2: Pré-processar os Comentários Raspados**

Agora, aplique a função `preprocess_text` que criamos para limpar os conteúdos dos reviews raspados. Você pode experimentar diferentes configurações (remover ou não números, pontuação, stop words, usar stemming).

In [26]:
# Utilize a função preprocess_text definida anteriormente

processed_reviews = []

if review_contents:
    print("Pré-processando os reviews com NLTK (Stemming)...")

    # Escolha suas opções de pré-processamento
    preprocess_options = {
        'remove_numbers': True,
        'remove_punctuation': True,
        'remove_stopwords': True,
        'apply_stemming': True # Experimente True e False
    }

    for i, review_text in enumerate(review_contents):
        if (i + 1) % 50 == 0: # Apenas para mostrar o progresso a cada 50 reviews
            print(f"  Processando review {i+1}/{len(review_contents)}...")

        # Aplica a função de pré-processamento
        if review_text and isinstance(review_text, str):
            processed_tokens = preprocess_text(review_text, **preprocess_options)
            processed_reviews.append(processed_tokens)
        else:
            # Adiciona uma lista vazia ou trata como preferir para reviews sem conteúdo
            processed_reviews.append([])


    print("Pré-processamento concluído!")

    # Exibir os tokens pré-processados dos primeiros 5 reviews (apenas se houver reviews processados)
    if processed_reviews:
        print("\nPrimeiros 5 reviews pré-processados (NLTK Stemming):")
        for i, tokens in enumerate(processed_reviews[:5]):
            print(f"--- Review {i+1} (Tokens) ---")
            print(tokens)
            print("----------------------------")
    else:
         print("\nNenhum review processado.")

else:
    print("Nenhum conteúdo de review para processar.")

Pré-processando os reviews com NLTK (Stemming)...
Pré-processamento concluído!

Primeiros 5 reviews pré-processados (NLTK Stemming):
--- Review 1 (Tokens) ---
['algum', 'bug', 'coment', 'vc', 'precis', 'envi', 'vár', 'vez', 'apag', 'nad', 'proib', 'car', 'emoj', 'feliz', 'fic', 'apag', 'anúnci', 'chat', 'algum', 'long', 'curt', 'aind', 'dá', 'pra', 'aguent', 'vc', 'pul', 'vem', 'outr', 'anúnci', 'outr', 'outr', 'demor', 'além', 'diss', 'bug', 'estranh', 'avanç', 'seg', 'vc', 'tent', 'avanç', 'tr', 'parec', 'disc', 'arranh', 'dj', 'arrum']
----------------------------
--- Review 2 (Tokens) ---
['bom', 'aqu', 'dnv', 'problem', 'popup', 'desd', 'temp', 'sid', 'resolv', 'dá', 'ver', 'agr', 'sugest', 'legal', 'aba', 'histór', 'acrescent', 'categor', 'exempl', 'assist', 'clip', 'músic', 'ent', 'fic', 'categor', 'músic', 'assist', 'víde', 'sobr', 'jog', 'algum', 'jog', 'aí', 'fic', 'categor', 'jog', 'tip', 'tent', 'ach', 'víde', 'notíc', 'víde', 'clip', 'histór', 'extens', 'bom', 'organiz']
-

### **Exercício 3: Análise Básica - Contagem de Frequência de Palavras**

Com os tokens pré-processados, uma análise básica interessante é contar a frequência de cada palavra. Quais são as palavras mais comuns nos comentários após a limpeza? Isso pode dar insights sobre os tópicos mais mencionados nos reviews.

In [27]:
from collections import Counter # Já importado no início
import itertools # Já importado no início

if processed_reviews:
    print("\n--- Análise de Frequência (NLTK Stemming) ---")
    # Achatar a lista de listas de tokens em uma única lista
    all_tokens_nltk_stem = list(itertools.chain.from_iterable(filter(None, processed_reviews)))
    print(f"Total de tokens (NLTK Stemming): {len(all_tokens_nltk_stem)}")

    if all_tokens_nltk_stem:
        token_counts_nltk_stem = Counter(all_tokens_nltk_stem)

        # Exibir os 30 tokens mais comuns
        print("\n30 tokens mais comuns (NLTK Stemming):")
        for token, count in token_counts_nltk_stem.most_common(30):
            print(f"'{token}': {count}")
    else:
         print("Nenhum token para contar (NLTK Stemming).")
else:
    print("\nNenhum review processado para contar a frequência.")


--- Análise de Frequência (NLTK Stemming) ---
Total de tokens (NLTK Stemming): 891

30 tokens mais comuns (NLTK Stemming):
'víde': 45
'anúnci': 18
'bug': 15
'algum': 13
'app': 13
'tel': 13
'pra': 12
'fic': 11
'youtub': 10
'dá': 9
'outr': 9
'sempr': 8
'atual': 8
'ver': 7
'assist': 7
'paus': 7
'aplic': 7
'tod': 7
'nov': 7
'vez': 6
'pul': 6
'temp': 6
'músic': 6
'gost': 6
'us': 6
'aparec': 6
'coloc': 6
'simples': 6
'chei': 6
'histór': 5
